In [8]:
import time
import os
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
from IPython.display import display, HTML
from IPython.display import Image
from urllib.parse import urljoin


In [3]:
df = pd.read_csv('final_corpus.csv')

In [11]:
def slow_scroll(url, close_driver=True):
    
    driver = webdriver.Chrome()

    # Open the webpage using the browser
    driver.get(url)


    SCROLL_PAUSE_TIME = 0.05

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    scroll_dist = 0
    while True:
        scroll_dist += 100
        # Scroll down to bottom
        driver.execute_script(f"window.scrollTo(0, {scroll_dist});")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if scroll_dist < new_height:
            continue

        if new_height == last_height:
            break
        last_height = new_height

    # Wait for the page to fully load (you might need to adjust the time)
    driver.implicitly_wait(10)  # Wait for up to 10 seconds

    # Get the page source after JavaScript execution
    page_source = driver.page_source
    
    if close_driver:
        driver.quit()
    return page_source

In [57]:
def parse_wapo(page_source, outdir, prefix):
    
    soup = BeautifulSoup(page_source, "html.parser")
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
        
        
    article_text = []
    title = soup.find('h1')
    title_text = title.get_text()
    article_text.append(title_text)
    
    # Find all img elements within the article content
    txt_elements = soup.select('[class*="copy"]')

    # Display the images inline in the notebook
    for txt in txt_elements:
        text = txt.get_text()
        article_text.append(text)
        
    
        
    article_text = '\n'.join(article_text)
    
    text_filename = f'{prefix:03d}_text.txt'
    
    with open(os.path.join(outdir, text_filename), 'w') as file:
        file.write(article_text)
        
    images = []
    
    # Find all img elements within the article content
    img_elements = soup.find_all("img")

    # Grab url
    for img in img_elements:
        img_src = img.get("src")
        
        if img_src is None:
            if img.get('srcset') is None:
                continue
            else:
                img_src = img.get('srcset')

        if img_src[:4]=='http':
            images.append(img_src)
        else:
            absolute_img_src = urljoin(url, img_src)
            images.append(absolute_img_src)
        
    for i,img_url in enumerate(images):
        print(f'request image {prefix}_{i}')
        response = requests.get(img_url, stream=True)
        
        try:
            image_obj = Image.open(response.raw)
            
            filename = f'{prefix:03d}_{i:02d}.png'

            image_obj.save(os.path.join(outdir, filename))
        except:
            continue
        
        
    
    
    

In [ ]:
start_idx = 11
for r, row in df.iterrows():
    
    if r < start_idx:
        continue
    
    outlet = row['Outlet']
    url = row['Article']
    
    if outlet == 'WaPo':
        
        page_source = slow_scroll(url, close_driver=False)
        print(f'scrolled {r}')
        
        parse_wapo(page_source,'./results/WaPo',r)


scrolled 13
request image 13_0
request image 13_1
request image 13_2
request image 13_3
request image 13_4
request image 13_5
request image 13_6
request image 13_7
request image 13_8
request image 13_9
request image 13_10
scrolled 14
request image 14_0
request image 14_1
request image 14_2
request image 14_3
request image 14_4
request image 14_5
request image 14_6
request image 14_7
request image 14_8
request image 14_9
request image 14_10
request image 14_11
request image 14_12
request image 14_13
request image 14_14
request image 14_15
request image 14_16
request image 14_17
request image 14_18
request image 14_19
request image 14_20
request image 14_21
request image 14_22
request image 14_23
request image 14_24
request image 14_25
request image 14_26
request image 14_27
request image 14_28
scrolled 15
request image 15_0
request image 15_1


In [15]:
from PIL import Image

In [20]:
parse_wapo(page_source,1,2)

TypeError: 'NoneType' object is not subscriptable

In [47]:
page_source

'<html lang="en" style="height: 100vh; overflow: hidden;"><head><meta charset="utf-8"><link rel="icon" href="https://www.washingtonpost.com/favicon.svg" type="image/svg+xml"><link rel="icon" href="https://www.washingtonpost.com/touch-icon-iphone.png" type="image/png"><link rel="apple-touch-icon" href="https://www.washingtonpost.com/touch-icon-iphone.png"><link rel="apple-touch-icon" sizes="152x152" href="https://www.washingtonpost.com/touch-icon-ipad.png"><link rel="apple-touch-icon" sizes="167x167" href="https://www.washingtonpost.com/touch-icon-ipad-retina.png"><link rel="apple-touch-icon" sizes="180x180" href="https://www.washingtonpost.com/touch-icon-iphone-retina.png"><title>The reality of coronavirus testing continues to differ from Trump’s claims - The Washington Post</title><meta name="description" content="Despite the president’s rhetoric, South Korea is sending testing kits to the United States."><meta property="og:site_name" content="Washington Post"><meta property="og:type"

In [34]:
    soup = BeautifulSoup(page_source, "html.parser")

    images = []
    
    # Find all img elements within the article content
    img_elements = soup.find_all("img")

    # Display the images inline in the notebook
    for img in img_elements:
        img_src = img.get("src")
        
        if img_src is None:
            if img.get('setsrc') is None:
                continue
            else:
                img_src = img.get('setsrc')

        if img_src[:4]=='http':
            images.append(img_src)
        else:
            absolute_img_src = urljoin(url, img_src)
            images.append(absolute_img_src)
        
    for i,img_url in enumerate(images):
        
        response = requests.get(img_url, stream=True)
        
        try:
            image_obj = Image.open(response.raw)

            image_obj.save(f'./test_img_{i}.png')
        except:
            continue

KeyboardInterrupt: 

In [28]:
img_src 

'https://img.washingtonpost.com/wp-apps/imrs.php?src=https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/5OUULWDQNUI6VIKWABELMLG3KE.jpg&w=1484&w=60&h=60'

In [30]:
response.raw

In [33]:
img_url 

'https://www.washingtonpost.com/resizer/ZP8LYU-5H-pq2ocspyx_XL3s2QA=/1200x630/filters:focal(759x427:1541x867):quality(100)/arc-anglerfish-washpost-prod-washpost/XQB62S46VJFZXP5FDFNABMPDEQ.jpg?c=5bb2f1c2d744a616473a8b543dba4ae8d5d385a7-1586371551'

In [44]:
url

'https://www.washingtonpost.com/politics/2020/04/13/reality-coronavirus-testing-continues-differ-trumps-claims/?utm_source=rss&utm_medium=referral&utm_campaign=wp_homepage'

In [48]:
soup = BeautifulSoup(page_source, "html.parser")

In [53]:
txt_elements = soup.select('[class*="copy"]')

In [54]:
txt_elements

[<p class="wpds-c-cYdRxM wpds-c-cYdRxM-iPJLV-css overrideStyles font-copy" data-el="text" data-testid="drop-cap-letter" dir="null">During the daily White House coronavirus press briefing early last week, President Trump praised the country’s testing regimen for the virus.</p>,
 <p class="wpds-c-cYdRxM wpds-c-cYdRxM-iPJLV-css overrideStyles font-copy" data-el="text" data-testid="drop-cap-letter" dir="null">“We have now done 1,790,000 tests nationwide,” Trump said. “That’s more than any other country in the world, hence we have more cases. And that number is growing by nearly 125,000 people per day. Think of that one. So it’s growing by 125,000 people per day.”</p>,
 <p class="wpds-c-cYdRxM wpds-c-cYdRxM-iPJLV-css overrideStyles font-copy" data-el="text" data-testid="drop-cap-letter" dir="null">“I told you about South Korea,” he added. “This is a, you know, vastly faster — and we also have, they say, the most accurate of tests.”</p>,
 <p class="wpds-c-cYdRxM wpds-c-cYdRxM-iPJLV-css overr